# 神经网络与tensorflow



## tensorflow介绍

TensorFlow 是一个用于人工智能的开源神器。

TensorFlow是一个采用数据流图（data flow graphs），用于数值计算的开源软件库。

它灵活的架构让你可以在多种平台上展开计算，例如台式计算机中的一个或多个CPU（或GPU），服务器，移动设备等等。

TensorFlow 最初由Google大脑小组的研究员和工程师们开发出来，用于机器学习和深度神经网络方面的研究，但这个系统的通用性使其也可广泛用于其他计算领域。


使用 TensorFlow, 你必须明白 TensorFlow:

- 使用图 (graph) 来表示计算任务；
- 图中的节点被称为op（operation）；
- 在被称之为 会话 (Session) 的上下文 (context) 中执行图；
- 使用 tensor (也叫张量、数组)表示数据；
- 每个tensor是一个类型化的多维数组；
  + 例如, 你可以将一小组图像集表示为一个四维浮点数数组, 这四个维度分别是[batch, height, width, channels].
  
- 一个节点可以获得0个或多个tensor（数据）进行计算，产生0或多个tensor;
- 通过变量 (Variable) 维护状态.
- 使用 feed 和 fetch 可以为任意的操作(arbitrary operation) 赋值或者从其中获取数据.



一个 TensorFlow 图描述了计算的过程. 为了进行计算, 图必须在会话里被启动。

会话将图的 op 分发到诸如 CPU 或 GPU 之类的设备上, 同时提供执行 op 的方法. 这些方法执行后, 将产生的 tensor 返回。

在 Python 语言中, 返回的 tensor 是 numpy ndarray 对象。

让我们先看一段使用 Python API 撰写的 TensorFlow 示例代码, 对将要学习的内容有初步的印象.


## 使用 tensorflow 实现神经网络学习器的过程

1. 准备数据;
2. 定义节点准备接收数据;
3. 定义隐藏层和输出层；
4. 定义损失函数；
5. 选定优化器，设定训练方法（使损失最小化）；
6. 对所有变量进行初始化，在会话中运行优化器，迭代1000次进行反馈学习。


### 定义神经网络层的构建方法

用tf.Variable定义变量。

In [2]:
import tensorflow as tf
import numpy as np

# 先写一个添加层的函数（可以一会儿再理解它）
def add_layer(inputs, in_size, out_size, activation_function=None):
    # 增加一个以上的层，并返回该层的输出。
    # 下面定义了神经元相关的权重、阈值、激活函数 
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))# 初始化连接权重
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)# 初始化阈值
    Wx_plus_b = tf.matmul(inputs, Weights) + biases #matmul指矩阵乘法,这里实际上定义了前向传播过程
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

### 准备数据

In [3]:
x_data = np.linspace(-1,1,300)[:, np.newaxis]# 生成300个数据，并将其变成2维数组
print(np.linspace(-1,1,300))
print(x_data)
noise = np.random.normal(0, 0.05, x_data.shape)
y_data = np.square(x_data) - 0.5 + noise

[-1.         -0.99331104 -0.98662207 -0.97993311 -0.97324415 -0.96655518
 -0.95986622 -0.95317726 -0.94648829 -0.93979933 -0.93311037 -0.9264214
 -0.91973244 -0.91304348 -0.90635452 -0.89966555 -0.89297659 -0.88628763
 -0.87959866 -0.8729097  -0.86622074 -0.85953177 -0.85284281 -0.84615385
 -0.83946488 -0.83277592 -0.82608696 -0.81939799 -0.81270903 -0.80602007
 -0.7993311  -0.79264214 -0.78595318 -0.77926421 -0.77257525 -0.76588629
 -0.75919732 -0.75250836 -0.7458194  -0.73913043 -0.73244147 -0.72575251
 -0.71906355 -0.71237458 -0.70568562 -0.69899666 -0.69230769 -0.68561873
 -0.67892977 -0.6722408  -0.66555184 -0.65886288 -0.65217391 -0.64548495
 -0.63879599 -0.63210702 -0.62541806 -0.6187291  -0.61204013 -0.60535117
 -0.59866221 -0.59197324 -0.58528428 -0.57859532 -0.57190635 -0.56521739
 -0.55852843 -0.55183946 -0.5451505  -0.53846154 -0.53177258 -0.52508361
 -0.51839465 -0.51170569 -0.50501672 -0.49832776 -0.4916388  -0.48494983
 -0.47826087 -0.47157191 -0.46488294 -0.45819398 -0.

### 定义节点准备接收数据

在 TensorFlow 中用placeholder 来描述等待输入的节点，需要指定类型。

在执行节点的时候用一个字典“喂”这些节点。相当于先把变量 hold 住，然后每次从外部传入data，注意 placeholder 和 feed_dict 是绑定用的。

In [11]:
# define placeholder for inputs to network  
xs = tf.placeholder(tf.float32, [None, 1])
ys = tf.placeholder(tf.float32, [None, 1])


Tensor("Placeholder_2:0", shape=(?, 1), dtype=float32)


### 定义隐藏层和预测层

In [10]:
# add hidden layer 输入值是 xs，在隐藏层有 10 个神经元   
l1 = add_layer(xs, 1, 10, activation_function=tf.nn.relu)
print(l1)
# add output layer 输入值是隐藏层 l1，在预测层输出 1 个结果
prediction = add_layer(l1, 10, 1, activation_function=None)
print(prediction)

Tensor("Relu_2:0", shape=(?, 10), dtype=float32)
Tensor("add_11:0", shape=(?, 1), dtype=float32)


### 定义损失函数

In [6]:
# the error between prediciton and real data    
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                     reduction_indices=[1]))

### 选择 optimizer 使 loss 达到最小    

这里选择梯度下降（ Gradient Descent）这个最基本的优化器（ Optimizer）。

神经网络的核心思想是让损失达到最小，所以训练步骤被直接定义为利用优化器使误差最小。

In [7]:
# 这一行定义了用什么方式去减少 loss，学习率是 0.1       
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

Instructions for updating:
Use tf.cast instead.


### 对所有变量进行初始化、调整

经过前面的定义，下面在运行模型前要初始化所有变量。

In [8]:
init = tf.initialize_all_variables()
sess = tf.Session()
# 上面定义的都没有运算，直到 sess.run 才会开始运算
sess.run(init)

# 迭代 1000 次学习，sess.run optimizer
for i in range(1000):
    # training train_step 和 loss 都是由 placeholder 定义的运算，所以这里要用 feed 传入参数
    sess.run(train_step, feed_dict={xs: x_data, ys: y_data})
    if i % 50 == 0:
        # 每50步打印输出一次，查看优化结果：损失函数的大小
        print(sess.run(loss, feed_dict={xs: x_data, ys: y_data}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.12465158
0.011637847
0.0074542393
0.0060863076
0.005356229
0.004768065
0.004239615
0.0038120167
0.0034752993
0.0032102198
0.0030381908
0.0029247121
0.0028487167
0.0027858324
0.002737039
0.0026898677
0.0026525361
0.0026250172
0.0026020193
0.002575664
